In [504]:
#importar librerias 
import pandas as pd 
import numpy as np
import nltk 

In [506]:
#Cargar la data 
df_solicitudes = pd.read_csv("../files/input/solicitudes_de_credito.csv", sep = ";", index_col = 0)
df_solicitudes 

,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,masculino,comercio,fabrica de,prado,3,10.0,13/07/2018,5000000,microempresarial
1,femenino,servicio,recreativo-educativo,barrio caicedo,1,9.0,30/10/2017,6000000,microempresarial
2,femenino,comercio,tienda,aranjuez,2,4.0,3/08/2017,7300000,microempresarial
3,Masculino,comercio,variedades,cabecera san cristobal,3,60.0,20/11/2017,7000000,microempresarial
4,femenino,comercio,fabrica de,robledo,2,7.0,23/02/2017,5000000,microempresarial
...,...,...,...,...,...,...,...,...,...
10915,masculino,agropecuaria,comidas rapidas,villatina,1,8.0,9/05/2018,7000000,microempresarial
10916,masculino,comercio,fabrica de arepas,villa hermosa,3,8.0,23/01/2017,7377000,microempresarial
10917,femenino,comercio,variedades,cabecera san cristobal,1,60.0,12/04/2018,7800000,microempresarial
10918,femenino,servicio,cafe e internet,manrique oriental,2,3.0,1/06/2017,7370000,microempresarial


In [507]:
df_solicitudes["barrio"].value_counts().sort_index

<bound method Series.sort_index of barrio
robledo                   1000
manrique central no. 1     475
san javier no.1            416
aranjuez                   382
belen                      381
                          ... 
los angeles                  1
centro-admtivo               1
Villa Del Socorro            1
Bombona No. 1                1
fatima                       1
Name: count, Length: 430, dtype: int64>

In [508]:
print("\n" + "=" * 60)
print("MÉTODO 1: CONTAR NaT POR COLUMNA")
print("=" * 60)

# Contar NaT en cada columna
nat_por_columna = df_solicitudes.isna().sum()
print("\nNúmero de NaT/NaN por columna:")
print(nat_por_columna)

# Filtrar solo columnas con NaT
columnas_con_nat = nat_por_columna[nat_por_columna > 0]
print("\nColumnas que tienen valores NaT/NaN:")
print(columnas_con_nat)

print("\n" + "=" * 60)
print("MÉTODO 2: VER FILAS CON NaT")
print("=" * 60)

# Ver todas las filas que tienen al menos un NaT
filas_con_nat = df_solicitudes[df_solicitudes.isna().any(axis=1)]
print("\nFilas que contienen al menos un NaT/NaN:")
print(filas_con_nat)

# Ver filas con NaT en una columna específica
print("\nFilas con NaT en 'fecha1':")
print(df_solicitudes[df_solicitudes['fecha_de_beneficio'].isna()])


MÉTODO 1: CONTAR NaT POR COLUMNA

Número de NaT/NaN por columna:
sexo                        0
tipo_de_emprendimiento    102
idea_negocio                0
barrio                    102
estrato                     0
comuna_ciudadano            0
fecha_de_beneficio          0
monto_del_credito           0
línea_credito               0
dtype: int64

Columnas que tienen valores NaT/NaN:
tipo_de_emprendimiento    102
barrio                    102
dtype: int64

MÉTODO 2: VER FILAS CON NaT

Filas que contienen al menos un NaT/NaN:
            sexo tipo_de_emprendimiento     idea_negocio    barrio  estrato  \
48      femenino               servicio        papeleria       NaN        2   
78      femenino               comercio        papeleria       NaN        2   
107     femenino              industria      fabrica de        NaN        2   
197     femenino               servicio      restaurante       NaN        3   
239     femenino                    NaN       variedades  castilla        

In [509]:
#Conversion y limpieza de la columna monto_del_credito

df_solicitudes['monto_del_credito'] = df_solicitudes['monto_del_credito'].str.strip()
df_solicitudes['monto_del_credito'] = df_solicitudes['monto_del_credito'].str.replace(",", "")
df_solicitudes['monto_del_credito'] = df_solicitudes['monto_del_credito'].str.replace("$", "")
df_solicitudes['monto_del_credito'] = pd.to_numeric(df_solicitudes['monto_del_credito'], errors='coerce')

In [510]:
#Conversion y limpieza de la columa fecha
print("\n" + "=" * 60)
print("MÉTODO 4: FUNCIÓN ROBUSTA Y REUTILIZABLE")
print("=" * 60)

def convertir_fecha_robusta(df, columna, formatos=None):

    if formatos is None:
        formatos = [
            '%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d',
            '%d-%m-%Y', '%Y.%m.%d', '%b %d %Y', '%B %d, %Y',
            '%d %b %Y', '%d-%b-%Y', '%Y%m%d'
        ]
    
    nombre_nueva_columna = f"{columna}_convertida"
    
    # Inicializar con NaT
    df[nombre_nueva_columna] = pd.NaT
    
    conversiones = {}
    
    for formato in formatos:
        # Máscara de valores no convertidos
        pendientes = df[nombre_nueva_columna].isna()
        
        if pendientes.any():
            # Intentar conversión
            try:
                temp = pd.to_datetime(df.loc[pendientes, columna], 
                                     format=formato, 
                                     errors='coerce')
                
                exitos = temp.notna().sum()
                if exitos > 0:
                    df.loc[pendientes, nombre_nueva_columna] = temp
                    conversiones[formato] = exitos
                    
            except Exception as e:
                continue
    
    # Reporte
    total_convertidos = df[nombre_nueva_columna].notna().sum()
    total_filas = len(df)
    
    print(f"\n📊 Reporte de conversión para '{columna}':")
    print(f"   Total de filas: {total_filas}")
    print(f"   Convertidas exitosamente: {total_convertidos} ({total_convertidos/total_filas*100:.1f}%)")
    print(f"   Fallidas: {total_filas - total_convertidos}")
    
    if conversiones:
        print(f"\n   Formatos utilizados:")
        for formato, count in conversiones.items():
            print(f"      {formato}: {count} conversiones")
    
    # Identificar valores no convertidos
    no_convertidos = df[df[nombre_nueva_columna].isna()][columna].unique()
    if len(no_convertidos) > 0:
        print(f"\n   ⚠️  Valores no convertidos: {list(no_convertidos)}")
    
    return df

# Aplicar función robusta
df_resultado = convertir_fecha_robusta(df_solicitudes.copy(), 'fecha_de_beneficio')
print("\nDataFrame resultante:")
print(df_resultado[['fecha_de_beneficio']])
df_solicitudes["fecha_de_beneficio"] = df_resultado["fecha_de_beneficio_convertida"]



MÉTODO 4: FUNCIÓN ROBUSTA Y REUTILIZABLE

📊 Reporte de conversión para 'fecha_de_beneficio':
   Total de filas: 10920
   Convertidas exitosamente: 10920 (100.0%)
   Fallidas: 0

   Formatos utilizados:
      %d/%m/%Y: 10227 conversiones
      %Y/%m/%d: 693 conversiones

DataFrame resultante:
      fecha_de_beneficio
0             13/07/2018
1             30/10/2017
2              3/08/2017
3             20/11/2017
4             23/02/2017
...                  ...
10915          9/05/2018
10916         23/01/2017
10917         12/04/2018
10918          1/06/2017
10919         23/07/2018

[10920 rows x 1 columns]


In [511]:
df_solicitudes.dtypes

sexo                              object
tipo_de_emprendimiento            object
idea_negocio                      object
barrio                            object
estrato                            int64
comuna_ciudadano                 float64
fecha_de_beneficio        datetime64[ns]
monto_del_credito                float64
línea_credito                     object
dtype: object

In [512]:
# Opción A: Eliminar filas con NaT en columnas datetime
df_sin_nat = df_solicitudes.dropna(subset=["tipo_de_emprendimiento", "barrio"])
print(f"\nDataFrame sin NaT en fechas: {len(df_sin_nat)} filas")
display(df_sin_nat)


DataFrame sin NaT en fechas: 10716 filas


,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,masculino,comercio,fabrica de,prado,3,10.0,2018-07-13,5000000.0,microempresarial
1,femenino,servicio,recreativo-educativo,barrio caicedo,1,9.0,2017-10-30,6000000.0,microempresarial
2,femenino,comercio,tienda,aranjuez,2,4.0,2017-08-03,7300000.0,microempresarial
3,Masculino,comercio,variedades,cabecera san cristobal,3,60.0,2017-11-20,7000000.0,microempresarial
4,femenino,comercio,fabrica de,robledo,2,7.0,2017-02-23,5000000.0,microempresarial
...,...,...,...,...,...,...,...,...,...
10915,masculino,agropecuaria,comidas rapidas,villatina,1,8.0,2018-05-09,7000000.0,microempresarial
10916,masculino,comercio,fabrica de arepas,villa hermosa,3,8.0,2017-01-23,7377000.0,microempresarial
10917,femenino,comercio,variedades,cabecera san cristobal,1,60.0,2018-04-12,7800000.0,microempresarial
10918,femenino,servicio,cafe e internet,manrique oriental,2,3.0,2017-06-01,7370000.0,microempresarial


In [614]:
def limpiar_dataset_completo(df):
    df_copia = df.copy()
    
    # Paso 1: Normalizar texto
    columnas_string = ["sexo", "tipo_de_emprendimiento", "idea_negocio", "barrio","línea_credito"]
    print(columnas_string)
    
    for columna in columnas_string:
        df_copia[columna] = df_copia[columna].astype(str)
        df_copia[columna] = df_copia[columna].str.strip()
        df_copia[columna] = df_copia[columna].str.lower()
        df_copia[columna] = df_copia[columna].str.replace(" ", "")
        df_copia[columna] = df_copia[columna].str.translate(
            str.maketrans("", "", "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~¿?")
        )

    df_copia = df_copia.drop_duplicates(keep='first')
        
    return df_copia
df_prueba = df_solicitudes.copy()  
df_limpio_1 = limpiar_dataset_completo(df_sin_nat)
display(df_limpio_1)

        

['sexo', 'tipo_de_emprendimiento', 'idea_negocio', 'barrio', 'línea_credito']


,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,masculino,comercio,fabricade,prado,3,10.0,2018-07-13,5000000.0,microempresarial
1,femenino,servicio,recreativoeducativo,barriocaicedo,1,9.0,2017-10-30,6000000.0,microempresarial
2,femenino,comercio,tienda,aranjuez,2,4.0,2017-08-03,7300000.0,microempresarial
3,masculino,comercio,variedades,cabecerasancristobal,3,60.0,2017-11-20,7000000.0,microempresarial
4,femenino,comercio,fabricade,robledo,2,7.0,2017-02-23,5000000.0,microempresarial
...,...,...,...,...,...,...,...,...,...
10914,femenino,servicio,tienda,13denoviembre,1,8.0,2016-11-10,6890000.0,microempresarial
10915,masculino,agropecuaria,comidasrapidas,villatina,1,8.0,2018-05-09,7000000.0,microempresarial
10916,masculino,comercio,fabricadearepas,villahermosa,3,8.0,2017-01-23,7377000.0,microempresarial
10917,femenino,comercio,variedades,cabecerasancristobal,1,60.0,2018-04-12,7800000.0,microempresarial


In [618]:
df_limpio_1["barrio"].value_counts().sort_values()

barrio
barriodejesus           1
villanueva              1
barriocolombia          1
laaguacatala            1
lacuchilla              1
                     ... 
belen                 376
aranjuez              383
sanjavierno1          423
manriquecentralno1    483
robledo               990
Name: count, Length: 223, dtype: int64

In [619]:
from pathlib import Path

def save_data_corregida(df, output_path_str):

    # 1. Crear un objeto Path a partir de la cadena de ruta
    output_path = Path(output_path_str)
    
    # 2. Obtener el directorio padre (ej. '../files/output')
    directorio_salida = output_path.parent
    
    # 3. CREAR EL DIRECTORIO si no existe (la clave de la corrección)
    # 'parents=True' crea cualquier directorio padre necesario.
    # 'exist_ok=True' evita errores si el directorio ya existe.
    directorio_salida.mkdir(parents=True, exist_ok=True)
    
    # 4. Guardar el DataFrame
    df.to_csv(
        output_path, 
        index=True,
        sep = ";"
        )

# --- Uso de la función corregida ---

# Nota: Asumo que tu DataFrame se llama df_limpio (basado en la descripción)
# y NO df_limpio_1 (basado en tu código de ejemplo).
df_a_guardar = df_limpio_1

# La ruta que quieres usar
ruta_guardado = "../files/output/solicitudes_de_credito.csv"

# Llamada a la función
save_data_corregida(df_a_guardar, ruta_guardado)